# How I found a 38 billion dollar data error

Looking for Medicaid spending data, I stumbled upon duplicates that seemed to be doubling the State of Connecticut Comptroller's expenditure records.

Props to the comptroller's office for making these expenditures readily available, and to their quick response that it was in fact an error, and that they were working to fix it.

I downloaded the data Friday, July 21 and the erronious files are preserved in this repo in the [data/](data) folder.

In [1]:
import pandas as pd

In [2]:
def get_rev():
    
    df = pd.read_csv("data/revenue_budget.csv")
    
    return df

revdf = get_rev()

In [3]:
def get_exp():
    
    df = pd.read_csv("data/operating_budget.csv")
    
    return df

expdf = get_exp()

In [4]:
revdf.head()

,ledger_type,fiscal_year,description,account_descr,revenue_category,actual_amount,program_descr,fund_type,fund_descr
0,Revenue,2013,Other Revenue,Federal Aid Restricted,Non General Fund Revenue,18421.66,In-Service Training,Special Revenue,Special Revenue
1,Revenue,2012,Other Revenue,Teacher's Certificate Licenses,Licenses Permits and Fees,3776471.00,Revenue Deposits,General,General
2,Revenue,2013,Other Revenue,Federal Aid Restricted,Non General Fund Revenue,90524.54,Refugee Preventive Health,Special Revenue,Special Revenue
3,Revenue,2016,Other Revenue,Rest. Aid not Grant Transfer,Non General Fund Revenue,235000.00,Geospatial Info Sys DOT,Special Revenue,Special Revenue
4,Revenue,2017,Other Revenue,Federal Aid Restricted,Non General Fund Revenue,181383.97,Asbestos TSCA/OCM Prog Grant,Special Revenue,Special Revenue


In [61]:
### expdf.head()

In [20]:
expdf[
    (expdf["program"].str.contains("Medicaid"))
#     & (expdf["program"] == "Medicaid")
    & (expdf["program"].str.contains("Medicaid"))

    & (expdf["department"] == "Department of Social Services")
#     & (expdf["fund"] == "General Fund")
     ]

,ledger_type,fiscal_year,fund_type,fund,service,department,program,recommended_amount,actual_amount
2204,Expense,2016,General,General Fund,Human Services,Department of Social Services,Medicaid,2.469621e+09,2.391093e+09
2235,Expense,2016,Special Revenue,Federal & Other Restricted Act,Human Services,Department of Social Services,Medicaid Federal Share,3.711989e+09,3.603770e+09
2360,Expense,2016,Special Revenue,Federal & Other Restricted Act,Human Services,Department of Social Services,Medicaid Incentive Demo-MIPCD,6.283739e+06,1.409035e+06
3608,Expense,2016,Special Revenue,Federal & Other Restricted Act,Human Services,Department of Social Services,Adult Medicaid Quality Grant,1.207234e+06,6.186285e+05
4618,Expense,2017,Special Revenue,Federal & Other Restricted Act,Human Services,Department of Social Services,Medicaid Incentive Demo-MIPCD,3.433014e+05,3.433014e+05
5381,Expense,2016,Special Revenue,Federal & Other Restricted Act,Human Services,Department of Social Services,DSS UCONN Medicaid Partnership,2.000000e+05,0.000000e+00
5944,Expense,2013,General,General Fund,Human Services,Department of Social Services,Medicaid,0.000000e+00,0.000000e+00
5951,Expense,2013,Special Revenue,Federal & Other Restricted Act,Human Services,Department of Social Services,Medicaid Transformation,0.000000e+00,0.000000e+00
6863,Expense,2013,General,General Fund,Human Services,Department of Social Services,Medicaid-Other Long Term Care,1.758774e+08,1.748159e+08
7154,Expense,2012,General,General Fund,Human Services,Department of Social Services,Medicaid,4.796946e+09,4.714306e+09


# Ruh roh: $12 billion on Medicaid

Look below - if the data were right, spending on Medicaid (crudely just defined as anything with "Medicaid" in the name) went from 6.1 billion in 2016 to 12 billion in 2017. 

That looked more like a duplicate than real data though. Especially because the next table shows revenue pretty much the same from 2016 to 2017.

In [24]:
expdf[
    (expdf["program"].str.contains("Medicaid"))
    & (expdf["department"] == "Department of Social Services")
].groupby("fiscal_year").agg({"recommended_amount":sum,"actual_amount":sum})

,actual_amount,recommended_amount
fiscal_year,,
2012,4.714312e+09,4.797967e+09
2013,4.898532e+09,4.952604e+09
2014,5.445415e+09,5.542675e+09
2015,5.832508e+09,5.882979e+09
2016,5.996890e+09,6.189301e+09
2017,1.204951e+10,1.228077e+10


In [30]:
revdf[
    (revdf["program_descr"].str.contains("Medicaid"))
    #& (revdf["department"] == "Department of Social Services")
].groupby("fiscal_year").agg({"actual_amount":sum})

,actual_amount
fiscal_year,
2012,9.422486e+06
2013,6.535000e+05
2014,2.993958e+09
2015,3.484596e+09
2016,3.605798e+09
2017,3.618203e+09


# First duplicate test: Counting rows

The number of rows didn't double from 2016 to 2017, but it went up a lot. From around the 5,000ish ballpark in all previous years to 6,337 rows. 

In [31]:
import numpy as np

In [34]:
expdf.groupby("fiscal_year").agg({"actual_amount":np.size})

,actual_amount
fiscal_year,
2012,4701.0
2013,4775.0
2014,5025.0
2015,4875.0
2016,4774.0
2017,6337.0


In [36]:
len(expdf[expdf["fiscal_year"] == 2017])

6337

In [37]:
len(expdf[expdf["fiscal_year"] == 2016])

4774

# Dupe check 2: Looking for exact duplicate numbers

Another quick test -- since the dollar amounts are precise, I figured that there wouldn't be a ton of rows with the exact same dollar amount. Below I counted the number of values that appear exactly twice in the data set for each year. In all years before 2016 there were 25 or fewer, but in 2017 there were 1,653.

In [60]:
dupe_check = expdf.groupby(["fiscal_year","actual_amount"])\
.agg({"recommended_amount":np.size}).reset_index()

dupe_check[(dupe_check["recommended_amount"] == 2)]\
.groupby("fiscal_year").agg({"recommended_amount":np.size})

,recommended_amount
fiscal_year,
2012,19.0
2013,16.0
2014,21.0
2015,23.0
2016,12.0
2017,1653.0
